# Decodifica di Hamming

Nelle telecomunicazioni i canali su cui viaggia l'informazione non sempre sono ideali. La richiesta che un canale sia privo di rumore, ovvero che un messaggio ricevuto non abbia subito delle modifiche durante il suo viaggio e che sia rimasto uguale quello inviato, è una richiesta molto forte.

### Ridondanza

A livello logico l'informazione è data da una sequenza di bit e gli errori consistono in delle modifiche casuali nella sequenza. L'idea è quindi di inviare dell'informazione in eccesso rispetto a quella da trasmettere e sfruttare questa ridondanza per individuare e correggere gli errori. Un protocollo di questo tipo potrebbe consistere nell'inviare $2k+1$ volte lo stesso bit e in fase di decodifica correggere l'errore usando il bit in maggioranza, ma sarebbe molto costoso in termini di *overhead*.

### Parità

Dato un messaggio di $N$ bit $b_1, \cdots, b_N$, un *<u>parity-check</u>* consiste nell'aggiunge un bit $c_p$ che consente di rilevare la presenza di un qualsiasi numero <u>dispari</u> di errori.

$$ \left( c_p + \sum_{i=1}^{N} b_i \right) \; \% \; 2 = 0 $$

Seppur abbia un costo minimo in termini di memoria, il parity check consente di rilevare solamente la presenza di un numero dispari e non permette di correggerli.

## Codici di Hamming

I codici di Hamming sono una famiglia di codici a blocchi lineari che sfruttano la ridondanza e il controllo di parità per la correzione automatica degli errori.

> **Codice lineare**: codice di correzione degli errori per cui ogni combinazione lineare tramite una somma bit a bit di parole di codice (*code-words*) valide è ancora una parola di codice.

> **Codice a blocchi**: codice di correzione degli errori che codifica i dati in blocchi.

I codici di Hamming sono una famiglia di codici binari. Per ogni intero $r \ge 2$ c'è una code-word (un blocco) di lunghezza $n = 2^r - 1$ che contiene un messaggio di lunghezza $k = 2^r - r - 1$. Il *rate* di un codice di Hamming è $R = k / n = 1 - r / (2^r - 1)$.

### L'algoritmo

L'algoritmo di Hamming genera un codice *single-error correcting* (SEC) per un numero qualsiasi di bit.

Nella logica della decodifica è fondamentale considerare le posizioni dei bit in binario, quindi gli indici della sequenza $1, 2, 3, 4, \cdots$ diventeranno $1, 10, 11, 100, \cdots$ ad esempio i bit di controllo si troveranno in posizione $1$, $10$, $100$, ecc.

L'idea dell'algoritmo di Hamming è la seguente:

1. Le posizioni detite ai bit di parità sono potenze di due: $1, 10, 100, 1000, \cdots$. La particolarità delle potenze di due in binario è che sono sequenze dove il bit all'estrema sinistra è $1$ e gli altri sono $0$.

2. Le altre posizioni, indicizzate da sequenze con due o più bit $1$, contengono i bit di informazione.

3. Ogni bit di informazione è incluso in un insieme unico di $2$ o più bit di parità, determinato dalla forma binaria della sua posizione.

    1. Il bit di parità $1$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la posizione più a destra in una stringa binaria: $1, 11, 101, 111, \cdots$ ($1, 3, 5, 7, \cdots$).

    2. Il bit di parità $2$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la seconda posizione da destra in una stringa binaria: $10- 110, 110-111, 1010-1011 \cdots$ ($2-3, 6-7, 10-11 \cdots$).

    3. Il bit di parità $4$ copre tutte le posizioni di bit che hanno $1$ come bit che occupa la terza posizione da destra in una stringa binaria: $100-111, 1100-1111, 10100-10111 \cdots$ ($4-7, 12-15, 20-23 \cdots$).

    4. In generale ogni bit di parità copre tutti i bit dove l'operazione bit a bit AND tra la posizione del bit di parità e del bit di informazione è non nulla.

>L'operazione bit a bit AND, indicata con $\&$, esegue un confronto tra due variabili dando come risultato una terza variabile che presenta un $1$ in quelle posizioni in cui entrambe le variabili di partenza presentano $1$ e uno $0$ in tutte le altre.

```python
a = 60          # 00111100
b = 240         # 11110000
c = a & b = 48  # 00110000
```

4. Le posizioni dei bit di controllo sono state scelte in maniera tale che l'operazione bit a bit XOR tra tutte le posizioni dei bit che contengono $1$ sia nulla.

> L'operazione bit a bit XOR indicata con $\; \^{} \;$, esegue un confronto tra due variabili dando come risultato una terza variabile che presenta un $1$ in quelle posizioni in cui le due variabili di partenza presentano valori diversi e uno $0$ in tutte le altre posizioni.

```python
a = 60          # 00111100
b = 240         # 11110000
c = a ^ b = 48  # 11001100
```

5. Se il ricevitore riceve una stringa che ha lo XOR degli indici nullo significa che il messaggio non è stato corrotto, altrimenti lo XOR indica l'indice del bit corrotto. Il risultato di questa operazione si chiama *sindrome*.

### Il rumore

A livello logico i due comunicatori, che chiamiamo Alice e Bob, si stanno scambiando una sequenza di bit, che è la nostra informazione. Il messaggio inviato sarà dato da una sequenza $x_1, \cdots, x_n$ di bit $x_i \in \left\{0, 1\right\} \; \forall i = 1, \cdots, n$. Il rumore lo modellizziamo con una probabilità non nulla $p$ che avvenga un *bit-flip* su un simbolo $x_i$ inviato da Alice $p = P(x_i | y_i = 1 - x_i)$, dove $y_i$ è il simbolo letto da Bob.

Caratterizzazione del canale:

1. I bit non vengono dispersi: se nel canale entrano $N$, escono $N$ bit.

2. Ciascun bit viene corrotto con una probabilità

$$ P(a_i = 0 | b_i = 1) =  P(a_i = 1 | b_i = 0) = p_e \textrm{,}$$

che NON dipende dal tempo o dal messaggio ma solo dal canale.

3. In un canale "decente": $p_e \ll \frac{1}{2}$.

## Pseudocodice

## Implementazione

Messaggio che Alice vuole inviare a Bob

In [1]:
# importiamo le librerie necessarie
import random

In [2]:
# 'with' è un "gestore di contesto": garantisce che il file venga chiuso automaticamente non appena finiamo di leggere, evitando sprechi di memoria o errori di sistema
# questo comando apre il file "Alice.txt" in modalità lettura
with open("Alice.txt", "r") as file:
    # legge tutto il contenuto del file e rimuove eventuali spazi bianchi iniziali/finali
    input_str = file.read().strip() # stringa di input

print(input_str)

Ciao Bob!


Il messaggio viene prima convertito in codice ASCII, successivamente in codice binario

In [ ]:
# converte una stringa in una rappresentazione binaria, dove ogni carattere è rappresentato da 8 bit come lista di numeri
def string_to_binary(s):
    # per ogni carattere 'c' nella stringa, ottiene il codice ASCII con ord(c),
    # format(..., '08b') lo converte in binario con 8 bit con padding di zeri a sinistra, e crea una lista di interi
    # il ciclo più esterno (il primo) itera sui caratteri della stringa
    # il ciclo interno (il secondo) itera sui bit della stringa binaria di quel carattere e li converte in interi
    return [int(b) for c in s for b in format(ord(c), '08b')]

# lista che contiene il messaggio in rappresentazione binaria
input_bin = string_to_binary(input_str)

print(input_bin)

[0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]


Dividiamo la lista di bit in raggruppamenti di $k$ bit ciascuno, ognuno dei quali sarà completato da un numero $r$ di bit di parità che serviranno per il controllo degli errori secondo

$$ 2^r \ge k + r + 1 $$

più un bit di parità globale.
Nel blocco esteso al bit di parità sono riservate le posizioni indicizzate dalle potenze di $2$ (la parità globale è descritta dal bit in posizione $0$), il messaggio è scritto nei bit rimanenti.

In [ ]:
k = 4 # lunghezza del messaggio in bit per ogni blocco

# funzione che calcola le posizioni dei bit di parità
# prende in input la lunghezza del messaggio
def parity_bits(k):
    r = 0
    while (2**r) < (k + r + 1):
        r += 1
    return [2**i for i in range(r)]

# funzione che calcola le posizioni dei bit riservati al messaggio
# prende in input la lunghezza del messaggio
def message_bits(k):
    r = len(parity_bits(k)) # numero dei bit di parità
    data_bits = []
    for i in range(1, k + r + 1):   # l'elemento con indice 0 è riservato al bit di parità globale
        # verifica se 'i' non è una potenza di 2 usando l'operatore bitwise AND (&)
        # se i è potenza di 2 (es. 1, 2, 4, 8), 'i & (i-1)' è sempre 0
        if (i & (i - 1)) != 0:
            data_bits.append(i)
    return data_bits

r = len(parity_bits(k))  # numero dei bit di parità
n = 1 + k + r  # lunghezza del blocco (incluso il bit di parità globale)

# lista che conterrà i blocchi di dati da inviare (bit di messaggio + bit di parità)
sent_data = []

# ciclo sulle divisioni del messaggio originario
for i in range(0, int(len(input_bin)/k)):
    message = input_bin[i*k:(i+1)*k]  # estraiamo il blocco di k bit dal messaggio originale
    block = [0] * (n) 
    # inseriamo nelle loro posizioni: bit di messaggio, bit di parità, bit di parità globale
    message_idx = 0
    for i in message_bits(k):
        block[i] = message[message_idx]
        message_idx += 1
    for i in parity_bits(k):
        parity = 0
        for j in range(1, n):
            if j & i:  # Se il risultato dell'AND è diverso da zero, significa che la posizione 'j' è controllata dal bit di parità corrente
                parity ^= block[j]  # il comando '^' è lo XOR per calcolare la parità
        block[i] = parity
    # calcolo del bit di parità globale
    overall_parity = sum(block) % 2
    block[0] = overall_parity
    sent_data.append(block)

# dividi la lista "input_bin" in sottoliste di 4 bit ciascuna
input_bin_div = [input_bin[i:i+4] for i in range(0, len(input_bin), k)]
print(input_bin_div)
print(sent_data)

[[0, 1, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 0, 1], [0, 1, 1, 0], [1, 1, 1, 1], [0, 0, 1, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 1, 0], [1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]


Inseriamo degli errori casuali per ogni blocco che viene inviato a Bob

In [19]:
# introduciamo degli errori casuali
p = 0.05  # probabilità di errore (bit flip)
# send = 1
sent_data_with_errors = []
for i in range(len(sent_data)):
    block_with_errors = sent_data[i].copy()
    for j in range(len(sent_data[i])):
        if random.random() < p:
            # Effettua il bit flip
            if sent_data[i][j] == 0:
                block_with_errors[j] = 1
            else:
                block_with_errors[j] = 0
    sent_data_with_errors.append(block_with_errors)

print(sent_data)
print(sent_data_with_errors)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]


Proviamo a mandare a video il messaggio ricevuto da Bob con gli errori.

In [14]:
output_bin_with_errors = []

for i in range(len(sent_data_with_errors)):
    message = [sent_data_with_errors[i][j] for j in message_bits(k)]
    output_bin_with_errors.extend(message)
# converte una lista di bit (numeri 0 o 1) in una stringa alfanumerica
def binary_to_string(binary_list):
    # lista per raccogliere i caratteri convertiti
    result = []
    # itera sulla lista binaria a passi di 8 bit
    for i in range(0, len(binary_list), 8):
        # estrae un blocco di 8 bit
        byte = binary_list[i:i+8]
        # se il blocco ha esattamente 8 bit, lo converte
        if len(byte) == 8:
            # converte la lista in stringa binaria, poi in intero, poi in carattere ASCII
            byte_str = ''.join(str(b) for b in byte)
            char = chr(int(byte_str, 2))
            result.append(char)
    # unisce tutti i caratteri in una stringa
    return ''.join(result)

# output_str = binary_to_string(output_bin)
print(output_bin_with_errors)
output_str = binary_to_string(output_bin_with_errors)
print(output_str)

[0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Kian Rob!


Correggiamo gli errori.

In [20]:
# funzione che calcola la sindrome del blocco ricevuto
def sindrome(block):
    s = 0
    for i in range(1, len(block)):
        if block[i] == 1:
            s ^= i
    return s

# funzione che calcola il bit di parità globale
def global_parity_check(block):
    return sum(block) % 2

# funzione che rileva e corregge gli errori nel blocco ricevuto
def correct_errors(block):
    s = sindrome(block) # calcola la sindrome del blocco ricevuto
    if s == 0:
        return block  # nessun errore rilevato
    else:
        block[s] ^= 1  # Corregge l'errore
        if global_parity_check(block) == 0:
            return block  # errore singolo corretto
        else:
            return 1  # errore multiplo rilevato, chiediamo di reinviare il blocco

print(sent_data)
print(sent_data_with_errors)

sent_data_corrected = []
for i in range(len(sent_data_with_errors)):
    send = 1
    while send == 1:
        send = correct_errors(sent_data_with_errors[i])
    sent_data_corrected.append(send)

print(sent_data_corrected)

[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 0, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1]]
[[1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1,

In [21]:
output_bin_corrected = []

for i in range(len(sent_data_corrected)):
    message = [sent_data_corrected[i][j] for j in message_bits(k)]
    output_bin_corrected.extend(message)

output_str = binary_to_string(output_bin_corrected)
print(output_str)

Ciao Bob!
